In [ ]:
import os
import time
import sklearn
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay as CMD
from torch.utils.data import TensorDataset 
from torch.utils.data import DataLoader 
from torchvision import datasets, models, transforms
from google.colab import drive

# Connect to your google drive:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/MusterUe/src')

from utils import *


## Settings:
pretrained = True    
architecture = models.resnet18      # (funcion pointer)
learning_rate = 0.001
weight_decay = 0.1
batch_size = 32
device = 'cuda'                     # 'cuda' or 'cpu'

train_image = [3, 12]               # [row, column]
valid_image = [3, 13]               # [row, column]
test_image  = [2, 10]               # [row, column]

## Load Training Data

In [ ]:
class_names = {0: 'building', 1: 'car', 2: 'clutter'}   # label: name

crops_train, gt_train = load_crops(*train_image)
crops_valid, gt_valid = load_crops(*valid_image)

print("Number of input channels:      ", crops_train.shape[2])
print("Number of training samples:    ", crops_train.shape[-1])
print("Number of validation samples:  ", crops_valid.shape[-1])

# Plot 8 random samples (RGB):
plot_samples(crops_train, gt_train, class_names)

## Prepare Data for Network
Use `dataloaders` and `datasets` to **feed data mini-batch-wise** to network. Can also be used for data-augmentation, e.g. see `transforms` for `ImageFolder` dataset.

In [ ]:
def prep_data(images, labels, batch_size=64, shuffle=False):
  images_ = np.moveaxis(images, [0, 1, 2, 3], [2,3,1,0]) # (H,W,C,N) => (N,C​,H,W) 
  images_ = torch.tensor(images_).float()
  labels_ = torch.tensor(labels).squeeze().long()
  dataset = TensorDataset(images_, labels_)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
  return dataloader

dataloader_train = prep_data(crops_train, gt_train, batch_size=batch_size, shuffle=True)

#
# TODO: create validation dataloader
dataloader_valid = prep_data(crops_valid, gt_valid, batch_size=batch_size, shuffle=True)
#

## Create Network
**Download** (pretrained) network.
**Replace first layer** to accept `cin = 5 = [RGBIRnDSM]` input channels. **Replace last layer** to output probabilities for `cout = 3 = [building, car, clutter]` classes. 
We make sure that all layers use the same format (`float()`) and move the network to GPU.

In [ ]:
net = architecture(pretrained=True, progress=True)
# print(net) # Display network architecture
cin = crops_train.shape[2]
cout = len(class_names)

#
# TODO: replace first and last network layer.
#  - Find the names of those layers trough print(net)
#  - Redefine those layers with net.layername = torch.nn.XYZ(...)
#  - For the first layer (torch.nn.Conv2D), use the same parameters except the number of input channels (first parameter)
net.conv1 = torch.nn.Conv2d(cin, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#  - For the last layer (torch.nn.Linear), use the same parameters except out_features.
net.fc = torch.nn.Linear(in_features=512, out_features=cout, bias=True)
#


net.float();

#
# TODO: transfer network to the device devined in settings.
net.to(device)
#


## Training / Fine Tuning
`net.train()` to set network (-modules) in training mode or `net.eval()` for inference, which relevant e.g. with dropout and Batchnormalization.


In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
loss_fu = nn.CrossEntropyLoss()

log = []
tic = time.time()

for i in range(100):
  ## Training step
  net.train()
  rl = []   # running loss
  ra = []   # running accuracy
  for batch in dataloader_train:
    inputs = batch[0].to(device)
    labels = batch[1].to(device)
    #
    # TODO: training over each mini-batch (see demo script)
    optimizer.zero_grad()           # Set gradient buffers to zero to prevent accumulations
    outputs = net(inputs)           # Forward pass
    loss = loss_fu(outputs, labels) # Calc loss
    loss.backward()                 # Backpropagation: compute gradients
    optimizer.step()                # Parameter update
    # print("loss:", loss)
    # 

    oa = torch.sum(torch.argmax(outputs,1) == labels).item() / len(labels) * 100
    rl.append(loss.item())
    ra.append(oa)

  loss = np.mean(rl)
  acc = np.mean(ra)
  print("Epoch %2d,  loss: %.4f,  acc: %5.1f" % (i, loss, acc), end='')
  log.append({'loss': loss, 'acc': acc})

  ## Validation step
  net.eval()

  #
  # TODO: run validation set trough the network. Record loss and accuracy
  cl = []   # contrl loss
  ca = []   # control accuracy
  for batch in dataloader_valid:
    inputs = batch[0].to(device)
    labels = batch[1].to(device)
    outputs = net(inputs)           # Forward pass
    loss = loss_fu(outputs, labels) # Calc loss
  
    oa = torch.sum(torch.argmax(outputs,1) == labels).item() / len(labels) * 100
    cl.append(loss.item())
    ca.append(oa)

  loss = np.mean(cl)
  acc = np.mean(ca)
  #

  print(",  valid_loss: %.4f,  valid_acc: %5.1f" % (loss, acc))
  log[-1]['valid_loss'] = loss; 
  log[-1]['valid_acc'] = acc

print("Finished Training after:", time.time()-tic, "sec.")

In [ ]:
## Plotting Training Metrics:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
def p(ax, d, t):
  ax.plot(d)
  ax.set_title(t)
  ax.set_xlabel('epoch')
  ax.grid(True)
p(ax1, [x['loss'] for x in log], 'loss')
p(ax2, [x['acc'] for x in log], 'acc [%]')

#
# TODO: Plot valid metrics into same axes.
p(ax1, [x['valid_loss'] for x in log], 'loss')
p(ax2, [x['valid_acc'] for x in log], 'acc [%]')
#



plt.show()

## Testing
Inference on the test set

In [ ]:
net.eval()
with torch.no_grad():

  #
  # TODO: 
  #  - Import test crops & create dataloader
  crops_test, gt_test = load_crops(*test_image)
  dataloader_test = prep_data(crops_test, gt_test, batch_size=batch_size, shuffle=False)


  preds = torch.zeros(0).to(device)
  gt = torch.zeros(0, dtype=torch.int).to(device)

  probs = []

  for batch in dataloader_test:
    inputs = batch[0].to(device)
    gt = torch.cat((gt, batch[1].to(device)))
    outputs = net(inputs)
    preds = torch.cat((preds, nn.functional.softmax(outputs)))

probs, preds = torch.max(preds, 1)
probs = probs.detach().cpu()
preds = preds.detach().cpu()
gt = gt.detach().cpu()

  #  - Over all mini-batches:
  #     - Predict crop-wise network outputs for test dataset
  #     - collect the corresponding gt
  #  - Bring predictions & gt to cpu with .detach().cpu().
  #  - Use torch.nn.functional.softmax to convert the network outputs to probabilities.
  #  - Use torch.max to select per sample the class with the highest probability as prediction.
  #
  #  => variables (expected in further script):
  #        preds  = predicted class label per crop / bounding box.
  #        probs  = probabilitiy of predicted class label per crop / bounding box.
  #
  # Tips:
  #   - use torch.cat() to concatinate list of tensors (per mini batch) to a single tensor.
  #   - max, argmax = torch.max()
  #



## Evaluating Test Results

In [ ]:
# Overall Accuracy etc.:
acc       = sklearn.metrics.accuracy_score(gt, preds)
recall    = sklearn.metrics.recall_score(gt, preds, average=None)
precision = sklearn.metrics.precision_score(gt, preds, average=None)
f1        = sklearn.metrics.f1_score(gt, preds, average=None)
print("Overall Test Accuracy: %5.1f %%" % (acc*100))
print("Recall:    ", recall*100, '[%]')
print("Precision: ", precision*100, '[%]')
print("F1 score:  ", f1*100, '[%]')

# Confusion Matrix:
cm = confusion_matrix(gt, preds)
disp = CMD(confusion_matrix=cm, display_labels=list(class_names.values()))
disp = disp.plot(include_values=True, cmap='Greys', ax=None, xticks_rotation='horizontal')
plt.show()

## Plot Test Results
After loading the **original bounding box coordinates** and the **RGB test image**, plot predictions onto RGB image and save to drive.

In [ ]:
img = load_img(*test_image)
bbs = load_bbs(*test_image)

img_name = '%d_%d ' % tuple(test_image)
colors = ['b', 'y', 'r']
plot_bbs(img, bbs, gt,           colors=colors, title=img_name + ' GT')
plot_bbs(img, bbs, preds, probs, colors=colors, title=img_name + ' Prediction')